# NGP master catalogue: Flags

In [1]:
import numpy as np
from astropy.table import Table, Column

import itertools

from herschelhelp_internal.flagging import flag_outliers

In [2]:
SUFFIX = "20180219"
FIELD = "NGP"
catname = "../../dmu1/dmu1_ml_NGP/data/master_catalogue_ngp_{}.fits".format(SUFFIX)
master_catalogue = Table.read(catname)

In [3]:
g_bands = [           "GPC1 g", "90prime g"]
r_bands = [           "GPC1 r", "90prime r"]
z_bands = ["DECam z", "GPC1 z", "Mosaic z"]
y_bands = [           "GPC1 y",          "UKIDSS Y"]


all_bands = [g_bands, r_bands, z_bands, y_bands]

## 1. Magnitudes and magnitude erros

In [4]:
def flag_mag(flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue.add_column(Column(data=np.zeros(len(master_catalogue), dtype=bool), name=flagcol))
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [5]:
## dmu0: Pan-STARRS forced photometry cat 
gpc1_err = 0.0010860000038519502
bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
    flagcol_ap, flagcol_tot = "flag_ap_{}".format(basecol), "flag_{}".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
    mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)
    print('  Total magnitude')
    flag_mag(flagcol_tot, mask_tot)
    

GPC1 g
  Aperture magnitude
    Number of flagged objects: 1408
  Total magnitude
    Number of flagged objects: 1439
GPC1 r
  Aperture magnitude
    Number of flagged objects: 2057
  Total magnitude
    Number of flagged objects: 2161
GPC1 i
  Aperture magnitude
    Number of flagged objects: 2167
  Total magnitude
    Number of flagged objects: 2228
GPC1 z
  Aperture magnitude
    Number of flagged objects: 1556
  Total magnitude
    Number of flagged objects: 1562
GPC1 y
  Aperture magnitude
    Number of flagged objects: 641
  Total magnitude
    Number of flagged objects: 628


### 2.a DECaLS Total magnitudes

In [6]:
decam_mag = 14.999935
bands = ["DECam z"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    col = "m_{}".format(basecol)
    flagcol = "flag_{}".format(basecol)
    
    mask  = np.where((master_catalogue[col]  == decam_mag) | (master_catalogue[col] < 7))[0]
    
    print('  Total magnitude')
    flag_mag(flagcol, mask)

DECam z
  Total magnitude
    Number of flagged objects: 1


/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


## 2. Outliers

In [7]:
for band_of_a_kind in all_bands:
    for band1, band2 in itertools.combinations(band_of_a_kind, 2):
        #print(band1, band2)

        basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
        
        # Aperture mag
        col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
        ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
        flagcol1, flagcol2 = "flag_ap_{}".format(basecol1), "flag_ap_{}".format(basecol2)
        
        try:
            master_catalogue = flag_outliers(master_catalogue, col1, col2,
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
        except KeyError:
            print("One of {} and {} is not present in the catalogue.".format(col1, col2))
                      
        
        # Tot mag
        col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
        ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
        flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
              
        try:
            master_catalogue = flag_outliers(master_catalogue, col1, col2, 
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (total)".format(band1), "{} (total)".format(band2)))   
        except KeyError:
            print("One of {} and {} is not present in the catalogue.".format(col1, col2))          
                      

90prime g (aperture) - GPC1 g (aperture):
  Number of outliers: 1899
90prime g (total) - GPC1 g (total):
  Number of outliers: 316
90prime r (aperture) - GPC1 r (aperture):
  Number of outliers: 3898
90prime r (total) - GPC1 r (total):
  Number of outliers: 6
GPC1 z (aperture) - DECam z (aperture):
  Number of outliers: 8938
GPC1 z (total) - DECam z (total):
  Number of outliers: 5
Mosaic z (aperture) - DECam z (aperture):
  Number of outliers: 1202
Mosaic z (total) - DECam z (total):
  Number of outliers: 13
Mosaic z (aperture) - GPC1 z (aperture):
  Number of outliers: 3902
Mosaic z (total) - GPC1 z (total):
  Number of outliers: 0
UKIDSS Y (aperture) - GPC1 y (aperture):
  Number of outliers: 102
UKIDSS Y (total) - GPC1 y (total):
  Number of outliers: 291


## 3. Save table

In [ ]:
#Merge any aperture flags
for col in master_catalogue.colnames:
    if col.startswith("flag_ap_"):
        try:
            master_catalogue[col.replace("_ap_", "_")] = (master_catalogue[col.replace("_ap_", "_")] |
                                                          master_catalogue[col])
            master_catalogue.remove_column(col)
        except KeyError:
            print("{} only has aperture flags.".format(col))
            master_catalogue.rename_column(col, col.replace("_ap_", "_"))

In [8]:
flag_cols = ["help_id"]
for col in master_catalogue.colnames:
    if col.startswith("flag_"):
        flag_cols += [col]
new_catname = "./data/{}_{}_flags.fits".format(FIELD.lower(),SUFFIX)
master_catalogue[flag_cols].write(new_catname, overwrite = True)